In [4]:
!pip install joblib --quiet
print("Packages downloaded successfully")

Packages downloaded successfully


In [5]:
!conda config --add channels conda-forge

!conda config --set channel_priority strict
!conda install -c conda-forge auto-sklearn

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/zeus/miniconda3/envs/cloudspace

  added / updated specs:
    - auto-sklearn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    auto-sklearn-0.15.0        |     pyha804496_1         648 KB  conda-forge
    aws-c-auth-0.8.1           |       h205f482_0         106 KB  conda-forge
    aws-c-cal-0.8.1            |       h1a47875_3          46 KB  conda-forge
    aws-c-common-0.10.6        |       hb9d3cd8_0         231 KB  conda-forge
    aws-c-compression-0.3.0    |       h4e1184b_5          19 KB  conda-forge
    aws-c-event-stream-0.5.0   |      h7959bf6_11          53 KB  conda-forge
    aws-c-http-0.9.2           |       hefd7a92_4         193 KB  conda-forge
    aws-c-io-0.15.3            |       h173a860_6         154 KB  conda-forge
    aws-c-mqtt-0.11.

In [3]:
!conda config --show-sources


==> /teamspace/studios/this_studio/.condarc <==
changeps1: False
channel_priority: strict
channels:
  - conda-forge
  - defaults

==> envvars <==
always_yes: True



In [7]:
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'secrets'

In [ ]:
morgan_dataset = pd.read_csv("~/datasets/morgan_dataset.csv")
ap_dataset = pd.read_csv("~/datasets/ap_dataset.csv")
rdk5_dataset = pd.read_csv("~/datasets/rdk5_dataset.csv")

In [ ]:
# Ensure datasets are aligned (optional if already aligned)
morgan_dataset, ap_dataset = morgan_dataset.align(ap_dataset, join='inner', axis=0)
morgan_dataset, rdk5_dataset = morgan_dataset.align(rdk5_dataset, join='inner', axis=0)

# Split morgan_dataset into training and testing sets (80% train, 20% test)
morgan_train, morgan_test = train_test_split(morgan_dataset, test_size=0.2, random_state=42)

# Now split ap_dataset and rdk5_dataset based on the indices of the split
ap_train = ap_dataset.loc[morgan_train.index]  # Use the same indices from morgan_train
ap_test = ap_dataset.loc[morgan_test.index]  # Use the same indices from morgan_test

rdk5_train = rdk5_dataset.loc[morgan_train.index]  # Use the same indices from morgan_train
rdk5_test = rdk5_dataset.loc[morgan_test.index]  # Use the same indices from morgan_test

In [ ]:
# Split based on 'activity' as the target
X_morgan_train_activity = morgan_train.drop(columns=['pIC50'])
y_morgan_train_activity = morgan_train['activity']  # Target for activity

X_morgan_test_activity = morgan_test.drop(columns=['pIC50'])
y_morgan_test_activity = morgan_test['activity']  # Target for activity

X_ap_train_activity = ap_train.drop(columns=['pIC50'])
y_ap_train_activity = ap_train['activity']  # Target for activity

X_ap_test_activity = ap_test.drop(columns=['pIC50'])
y_ap_test_activity = ap_test['activity']  # Target for activity

X_rdk5_train_activity = rdk5_train.drop(columns=['pIC50'])
y_rdk5_train_activity = rdk5_train['activity']  # Target for activity

X_rdk5_test_activity = rdk5_test.drop(columns=['pIC50'])
y_rdk5_test_activity = rdk5_test['activity']  # Target for activity

# Split based on 'pIC50' as the target
X_morgan_train_pic50 = morgan_train.drop(columns=['activity'])
y_morgan_train_pic50 = morgan_train['pIC50']  # Target for pIC50

X_morgan_test_pic50 = morgan_test.drop(columns=['activity'])
y_morgan_test_pic50 = morgan_test['pIC50']  # Target for pIC50

X_ap_train_pic50 = ap_train.drop(columns=['activity'])
y_ap_train_pic50 = ap_train['pIC50']  # Target for pIC50

X_ap_test_pic50 = ap_test.drop(columns=['activity'])
y_ap_test_pic50 = ap_test['pIC50']  # Target for pIC50

X_rdk5_train_pic50 = rdk5_train.drop(columns=['activity'])
y_rdk5_train_pic50 = rdk5_train['pIC50']  # Target for pIC50

X_rdk5_test_pic50 = rdk5_test.drop(columns=['activity'])
y_rdk5_test_pic50 = rdk5_test['pIC50']  # Target for pIC50

In [ ]:
import autosklearn.classification
import autosklearn.regression
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
import joblib
import os
import pandas as pd
from datetime import datetime

def evaluate_and_save_results(results, dataset_name, task_type, save_path):
    """
    Save evaluation results to a CSV file.
    
    Args:
        results: List of dictionaries containing evaluation metrics
        dataset_name: Name of the dataset
        task_type: 'classification' or 'regression'
        save_path: Directory to save results
    """
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    
    # Create save directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)
    
    # Generate a timestamped filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{dataset_name}_{task_type}_results_{timestamp}.csv"
    filepath = os.path.join(save_path, filename)
    
    # Save to CSV
    results_df.to_csv(filepath, index=False)
    print(f"Results saved to {filepath}")

def train_autosklearn_classifier(X_train, y_train, X_test, y_test, dataset_name='morgan', 
                                time_limit=3600, memory_limit=4096, save_path=None):
    """
    Train an Auto-Sklearn classification model for activity prediction.
    
    Args:
        X_train, y_train: Training data and labels
        X_test, y_test: Test data and labels
        dataset_name: Name prefix for saving models
        time_limit: Training time limit in seconds
        memory_limit: Memory limit in MB
        save_path: Directory to save models
    
    Returns:
        Tuple: (trained_model, test_predictions, accuracy, f1)
    """
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=time_limit,
        memory_limit=memory_limit,
        n_jobs=-1,
        seed=42
    )
    
    automl.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = automl.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Save model
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        model_file = os.path.join(save_path, f'{dataset_name}_activity_classifier.pkl')
        joblib.dump(automl, model_file)
    
    return automl, y_pred, acc, f1

def train_autosklearn_regressor(X_train, y_train, X_test, y_test, dataset_name='morgan',
                               time_limit=3600, memory_limit=4096, save_path=None):
    """
    Train an Auto-Sklearn regression model for pIC50 prediction.
    
    Args:
        X_train, y_train: Training data and targets
        X_test, y_test: Test data and targets
        dataset_name: Name prefix for saving models
        time_limit: Training time limit in seconds
        memory_limit: Memory limit in MB
        save_path: Directory to save models
    
    Returns:
        Tuple: (trained_model, test_predictions, rmse, r2)
    """
    automl = autosklearn.regression.AutoSklearnRegressor(
        time_left_for_this_task=time_limit,
        memory_limit=memory_limit,
        n_jobs=-1,
        seed=42
    )
    
    automl.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = automl.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    # Save model
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        model_file = os.path.join(save_path, f'{dataset_name}_pic50_regressor.pkl')
        joblib.dump(automl, model_file)
    
    return automl, y_pred, rmse, r2

def run_autosklearn_experiments(X_train, y_train, X_test, y_test, dataset_name='morgan', 
                                task_type='classification', num_runs=10, time_limit=300, 
                                memory_limit=4096, save_path='results'):
    """
    Run Auto-Sklearn experiments multiple times and save results.
    
    Args:
        X_train, y_train: Training data and labels/targets
        X_test, y_test: Test data and labels/targets
        dataset_name: Name of the dataset
        task_type: 'classification' or 'regression'
        num_runs: Number of times to repeat the experiment
        time_limit: Training time limit in seconds
        memory_limit: Memory limit in MB
        save_path: Directory to save results
    """
    results = []
    
    for run in range(num_runs):
        print(f"Starting run {run + 1}/{num_runs} for {dataset_name} ({task_type})...")
        
        if task_type == 'classification':
            model, y_pred, acc, f1 = train_autosklearn_classifier(
                X_train, y_train, X_test, y_test, dataset_name, time_limit, memory_limit, save_path
            )
            results.append({
                'run': run + 1,
                'accuracy': acc,
                'f1_score': f1,
                'dataset': dataset_name,
                'task_type': task_type
            })
        elif task_type == 'regression':
            model, y_pred, rmse, r2 = train_autosklearn_regressor(
                X_train, y_train, X_test, y_test, dataset_name, time_limit, memory_limit, save_path
            )
            results.append({
                'run': run + 1,
                'rmse': rmse,
                'r2_score': r2,
                'dataset': dataset_name,
                'task_type': task_type
            })
        else:
            raise ValueError("task_type must be 'classification' or 'regression'")
    
    # Save evaluation results
    evaluate_and_save_results(results, dataset_name, task_type, save_path)

# Example usage:
# For activity classification (Morgan features)
run_autosklearn_experiments(
    X_train=X_morgan_train_activity,
    y_train=y_morgan_train_activity,
    X_test=X_morgan_test_activity,
    y_test=y_morgan_test_activity,
    dataset_name='morgan',
    task_type='classification',
    num_runs=10,
    save_path='autosklearn_results'
)

# For pIC50 regression (AP features)
run_autosklearn_experiments(
    X_train=X_ap_train_pic50,
    y_train=y_ap_train_pic50,
    X_test=X_ap_test_pic50,
    y_test=y_ap_test_pic50,
    dataset_name='ap',
    task_type='regression',
    num_runs=10,
    save_path='autosklearn_results'
)

ModuleNotFoundError: No module named 'autosklearn'

In [ ]:
# Define the save path
save_path = "datasets/autosklearn"

# Run experiments for 'activity' classification
run_autosklearn_experiments(
    X_train=X_morgan_train_activity,
    y_train=y_morgan_train_activity,
    X_test=X_morgan_test_activity,
    y_test=y_morgan_test_activity,
    dataset_name='morgan',
    task_type='classification',
    num_runs=10,
    save_path=save_path
)

In [ ]:
save_path = "datasets/autosklearn"

run_autosklearn_experiments(
    X_train=X_ap_train_activity,
    y_train=y_ap_train_activity,
    X_test=X_ap_test_activity,
    y_test=y_ap_test_activity,
    dataset_name='ap',
    task_type='classification',
    num_runs=10,
    save_path=save_path
)

In [ ]:
save_path = "datasets/autosklearn"

run_autosklearn_experiments(
    X_train=X_rdk5_train_activity,
    y_train=y_rdk5_train_activity,
    X_test=X_rdk5_test_activity,
    y_test=y_rdk5_test_activity,
    dataset_name='rdk5',
    task_type='classification',
    num_runs=10,
    save_path=save_path
)

In [ ]:
# Run experiments for 'pIC50' regression
save_path = "datasets/autosklearn"

run_autosklearn_experiments(
    X_train=X_morgan_train_pic50,
    y_train=y_morgan_train_pic50,
    X_test=X_morgan_test_pic50,
    y_test=y_morgan_test_pic50,
    dataset_name='morgan',
    task_type='regression',
    num_runs=10,
    save_path=save_path
)

In [ ]:
save_path = "datasets/autosklearn"

run_autosklearn_experiments(
    X_train=X_ap_train_pic50,
    y_train=y_ap_train_pic50,
    X_test=X_ap_test_pic50,
    y_test=y_ap_test_pic50,
    dataset_name='ap',
    task_type='regression',
    num_runs=10,
    save_path=save_path
)


In [ ]:
save_path = "datasets/autosklearn"

run_autosklearn_experiments(
    X_train=X_rdk5_train_pic50,
    y_train=y_rdk5_train_pic50,
    X_test=X_rdk5_test_pic50,
    y_test=y_rdk5_test_pic50,
    dataset_name='rdk5',
    task_type='regression',
    num_runs=10,
    save_path=save_path
)